# <font color=pink>**Extração de dados do FMI usando Python**</font>

In [2]:
# Imports
import requests
import pandas as pd
import json

Referência: https://datahelp.imf.org/knowledgebase/articles/667681-using-json-restful-web-service

In [3]:
# Request para ver a lista de datasets disponíveis na API no FMI
url = "http://dataservices.imf.org/REST/SDMX_JSON.svc/Dataflow"
response = requests.get(url).json()

# Formatando o json
json_formatado = json.dumps(response, indent=4, sort_keys=True)
print(json_formatado)

{
    "Structure": {
        "@xmlns": "http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message",
        "@xmlns:xsd": "http://www.w3.org/2001/XMLSchema",
        "@xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance",
        "@xsi:schemaLocation": "http://www.SDMX.org/resources/SDMXML/schemas/v2_0/structure https://registry.sdmx.org/schemas/v2_0/SDMXStructure.xsd http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message https://registry.sdmx.org/schemas/v2_0/SDMXMessage.xsd",
        "Dataflows": {
            "Dataflow": [
                {
                    "@agencyID": "IMF",
                    "@id": "DS-BOP_2017M06",
                    "@isFinal": "true",
                    "@version": "1.0",
                    "@xmlns": "http://www.SDMX.org/resources/SDMXML/schemas/v2_0/structure",
                    "KeyFamilyRef": {
                        "KeyFamilyAgencyID": "IMF",
                        "KeyFamilyID": "BOP_2017M06"
                    },
                    "

In [9]:
# Acessando a chave 'Dataflows' para acessar os dados dentro dela
bases = response['Structure']['Dataflows']['Dataflow']

lista_bases = []

# Iterando sobre a lista de bases de dados
for base in bases:
    dataset_info = {
        'ID': base['@id'],
        'Name': base['Name']['#text']
    }
    lista_bases.append(dataset_info)

# Transformando em df
df = pd.DataFrame(lista_bases)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
#pd.reset_option('display.max_rows')
#pd.reset_option('display.max_columns')
df.sort_values(by="Name")

,ID,Name
236,DS-APDREO,Asia and Pacific Regional Economic Outlook (AP...
158,DS-APDREO201504,Asia and Pacific Regional Economic Outlook (AP...
151,DS-APDREO201604,Asia and Pacific Regional Economic Outlook (AP...
101,DS-APDREO2017M04,Asia and Pacific Regional Economic Outlook (AP...
58,DS-APDREO201804,Asia and Pacific Regional Economic Outlook (AP...
159,DS-APDREO201904,Asia and Pacific Regional Economic Outlook (AP...
73,DS-APDREO201410,Asia and Pacific Regional Economic Outlook (AP...
94,DS-APDREO201510,Asia and Pacific Regional Economic Outlook (AP...
82,DS-APDREO201610,Asia and Pacific Regional Economic Outlook (AP...
117,DS-APDREO201710,Asia and Pacific Regional Economic Outlook (AP...


In [ ]:
# Endpoint da API para ver estrutura de dados do dataset Financial Access Survey (FAS)
database_id = "FAS"
url = f"http://dataservices.imf.org/REST/SDMX_JSON.svc/DataStructure/{database_id}"

# Request
response = requests.get(url).json()

# Formatando json
json_formatado = json.dumps(response, indent=4, sort_keys=True)
print(json_formatado)

{
    "Structure": {
        "@xmlns": "http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message",
        "@xmlns:xsd": "http://www.w3.org/2001/XMLSchema",
        "@xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance",
        "@xsi:schemaLocation": "http://www.SDMX.org/resources/SDMXML/schemas/v2_0/structure https://registry.sdmx.org/schemas/v2_0/SDMXStructure.xsd http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message https://registry.sdmx.org/schemas/v2_0/SDMXMessage.xsd",
        "CodeLists": {
            "CodeList": [
                {
                    "@agencyID": "IMF",
                    "@id": "CL_UNIT_MULT",
                    "@isFinal": "true",
                    "@version": "1.0",
                    "@xmlns": "http://www.SDMX.org/resources/SDMXML/schemas/v2_0/structure",
                    "Code": [
                        {
                            "@value": "0",
                            "Description": {
                                "#text": "Uni

In [ ]:
# Endpoint da API para acessar os indicadores que vamos usar (referentes às mulheres)
frequency = "A"  # período anual
items_dimension1 = "BR"  # país Brasil
items_dimension2_F = "FCRAMFHFFA_NUM+FCRODCHFFA_NUM+FCDODCHFFA_NUM+FCAODCHFFA_NUM+FCNAMFHFFA_NUM+FCNODCHFFA_NUM+FCRODCHF_PE_NUM+FCROFNMFHF_NUM+FCDODCHF_PE_NUM"  # indicadores
url = f"http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/{database_id}/{frequency}.{items_dimension1}.{items_dimension2_F}"

# Request
response = requests.get(url).json()

# Formatando json
json_formatado = json.dumps(response, indent=4, sort_keys=True)
print(json_formatado)

{
    "CompactData": {
        "@xmlns": "http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message",
        "@xmlns:xsd": "http://www.w3.org/2001/XMLSchema",
        "@xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance",
        "@xsi:schemaLocation": "http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message https://registry.sdmx.org/schemas/v2_0/SDMXMessage.xsd http://dataservices.imf.org/compact/FAS http://dataservices.imf.org/compact/FAS.xsd",
        "DataSet": {
            "@xmlns": "http://dataservices.imf.org/compact/FAS",
            "Series": [
                {
                    "@FREQ": "A",
                    "@INDICATOR": "FCRODCHF_PE_NUM",
                    "@REF_AREA": "BR",
                    "@TIME_FORMAT": "P1Y",
                    "@UNIT_MULT": "0",
                    "Obs": [
                        {
                            "@OBS_VALUE": "41978627",
                            "@TIME_PERIOD": "2018"
                        },
                   

In [ ]:
# Acessando a chave 'Series' para acessar a lista de indicadores
indicadores = response['CompactData']['DataSet']['Series']

# Iterando sobre as listas de indicadores
lista_indicadores = []
for indicador in indicadores:
    for obs in indicador['Obs']:
        dataset_info = {
            'Código do indicador': indicador['@INDICATOR'],
            'Localização': indicador['@REF_AREA'],
            'Período': obs['@TIME_PERIOD'],
            'Valor': obs['@OBS_VALUE']
        }
        lista_indicadores.append(dataset_info)

# Transformando em um df
df1 = pd.DataFrame(lista_indicadores)
df1

,Código do indicador,Localização,Período,Valor
0,FCRODCHF_PE_NUM,BR,2018,41978627
1,FCRODCHF_PE_NUM,BR,2019,50952985
2,FCRODCHF_PE_NUM,BR,2020,52385126
3,FCRODCHF_PE_NUM,BR,2021,55718475
4,FCRODCHF_PE_NUM,BR,2022,59134360
5,FCRODCHFFA_NUM,BR,2018,495.47627639123
6,FCRODCHFFA_NUM,BR,2019,594.460228250488
7,FCRODCHFFA_NUM,BR,2020,604.584115740931
8,FCRODCHFFA_NUM,BR,2021,637.076208954367
9,FCRODCHFFA_NUM,BR,2022,670.520208667104


In [ ]:
# Endpoint da API para acessar os indicadores que vamos analisar (referentes aos homens)
items_dimension2_M = "FCROFNMFHM_NUM+FCDODCHM_PE_NUM+FCRAMFHMMA_NUM+FCRODCHMMA_NUM+FCDODCHMMA_NUM+FCAODCHMMA_NUM+FCNAMFHMMA_NUM+FCNODCHMMA_NUM+FCRODCHM_PE_NUM"  # indicadores
url = f"http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/{database_id}/{frequency}.{items_dimension1}.{items_dimension2_M}"

# Request
response = requests.get(url).json()

# Acessando a chave 'Series' para acessar a lista de indicadores
indicadores = response['CompactData']['DataSet']['Series']

# Iterando sobre as listas de indicadores
lista_indicadores = []
for indicador in indicadores:
    for obs in indicador['Obs']:
        dataset_info = {
            'Código do indicador': indicador['@INDICATOR'],
            'Localização': indicador['@REF_AREA'],
            'Período': obs['@TIME_PERIOD'],
            'Valor': obs['@OBS_VALUE']
        }
        lista_indicadores.append(dataset_info)

# Transformando em df
df2 = pd.DataFrame(lista_indicadores)
df2

,Código do indicador,Localização,Período,Valor
0,FCRODCHM_PE_NUM,BR,2018,43965531
1,FCRODCHM_PE_NUM,BR,2019,52348853
2,FCRODCHM_PE_NUM,BR,2020,53516588
3,FCRODCHM_PE_NUM,BR,2021,56461759
4,FCRODCHM_PE_NUM,BR,2022,58856297
5,FCRODCHMMA_NUM,BR,2018,546.947515505523
6,FCRODCHMMA_NUM,BR,2019,643.991648321729
7,FCRODCHMMA_NUM,BR,2020,651.601980129402
8,FCRODCHMMA_NUM,BR,2021,681.555338082682
9,FCRODCHMMA_NUM,BR,2022,705.078048161753


In [ ]:
indicador_nomes = {
    'FCRAMFHFFA_NUM': 'Number of female borrowers of the household sector from all microfinance institutions per 1,000 female adults',
    'FCRAMFHMMA_NUM': 'Number of male borrowers of the household sector from all microfinance institutions per 1,000 male adults',
    'FCRODCHFFA_NUM': 'Number of female borrowers of the household sector from commercial banks per 1,000 female adult',
    'FCRODCHMMA_NUM': 'Number of male borrowers of the household sector from commercial banks per 1,000 male adult',
    'FCDODCHFFA_NUM': 'Number of female depositors of the household sector with commercial banks per 1,000 female adults',
    'FCDODCHMMA_NUM': 'Number of male depositors of the household sector with commercial banks per 1,000 male adults',
    'FCAODCHFFA_NUM': 'Number of women-owned deposit accounts of the household sector with commercial banks per 1,000 female adults',
    'FCAODCHMMA_NUM': 'Number of men-owned deposit accounts of the household sector with commercial banks per 1,000 male adults',
    'FCNAMFHFFA_NUM': 'Number of women-owned loan accounts of the household sector with all microfinance institutions per 1,000 female adults',
    'FCNAMFHMMA_NUM': 'Number of men-owned loan accounts of the household sector with all microfinance institutions per 1,000 male adults',
    'FCNODCHFFA_NUM': 'Number of women-owned loan accounts of the household sector with commercial banks per 1,000 female adults',
    'FCNODCHMMA_NUM': 'Number of men-owned loan accounts of the household sector with commercial banks per 1,000 male adults',
    'FCRODCHF_PE_NUM': 'Number of Borrowers, Commercial banks, of which: household sector borrowers, of which: female borrowers',
    'FCRODCHM_PE_NUM': 'Number of Borrowers, Commercial banks, of which: household sector borrowers, of which: male borrowers',
    'FCROFNMFHF_NUM': 'Non-deposit taking microfinance institutions, of which: household sector borrowers, of which: female borrowers',
    'FCROFNMFHM_NUM': 'Non-deposit taking microfinance institutions, of which: household sector borrowers, of which: male borrowers',
    'FCDODCHF_PE_NUM': 'Commercial banks, of which: household sector depositors, of which: female depositors',
    'FCDODCHM_PE_NUM': 'Commercial banks, of which: household sector depositors, of which: male depositors'
}

# Criando nova coluna com o nome dos indicadores
df1['Nome do Indicador'] = df1['Código do indicador'].map(indicador_nomes)
df2['Nome do Indicador'] = df2['Código do indicador'].map(indicador_nomes)

In [ ]:
df1

,Código do indicador,Localização,Período,Valor,Nome do Indicador
0,FCRODCHF_PE_NUM,BR,2018,41978627,"Number of Borrowers, Commercial banks, of whic..."
1,FCRODCHF_PE_NUM,BR,2019,50952985,"Number of Borrowers, Commercial banks, of whic..."
2,FCRODCHF_PE_NUM,BR,2020,52385126,"Number of Borrowers, Commercial banks, of whic..."
3,FCRODCHF_PE_NUM,BR,2021,55718475,"Number of Borrowers, Commercial banks, of whic..."
4,FCRODCHF_PE_NUM,BR,2022,59134360,"Number of Borrowers, Commercial banks, of whic..."
5,FCRODCHFFA_NUM,BR,2018,495.47627639123,Number of female borrowers of the household se...
6,FCRODCHFFA_NUM,BR,2019,594.460228250488,Number of female borrowers of the household se...
7,FCRODCHFFA_NUM,BR,2020,604.584115740931,Number of female borrowers of the household se...
8,FCRODCHFFA_NUM,BR,2021,637.076208954367,Number of female borrowers of the household se...
9,FCRODCHFFA_NUM,BR,2022,670.520208667104,Number of female borrowers of the household se...


In [ ]:
df2

,Código do indicador,Localização,Período,Valor,Nome do Indicador
0,FCRODCHM_PE_NUM,BR,2018,43965531,"Number of Borrowers, Commercial banks, of whic..."
1,FCRODCHM_PE_NUM,BR,2019,52348853,"Number of Borrowers, Commercial banks, of whic..."
2,FCRODCHM_PE_NUM,BR,2020,53516588,"Number of Borrowers, Commercial banks, of whic..."
3,FCRODCHM_PE_NUM,BR,2021,56461759,"Number of Borrowers, Commercial banks, of whic..."
4,FCRODCHM_PE_NUM,BR,2022,58856297,"Number of Borrowers, Commercial banks, of whic..."
5,FCRODCHMMA_NUM,BR,2018,546.947515505523,Number of male borrowers of the household sect...
6,FCRODCHMMA_NUM,BR,2019,643.991648321729,Number of male borrowers of the household sect...
7,FCRODCHMMA_NUM,BR,2020,651.601980129402,Number of male borrowers of the household sect...
8,FCRODCHMMA_NUM,BR,2021,681.555338082682,Number of male borrowers of the household sect...
9,FCRODCHMMA_NUM,BR,2022,705.078048161753,Number of male borrowers of the household sect...
